In [5]:
from radprompter import RadPrompter, Prompt, OpenAIClient

In [6]:
SCHEMA = [
    {
        "name": "Pneumothorax",
        "type": "select",
        "options": ["Present", "Absent"],
        "hint": "Indicate `Present` if the report explicitly mentions the patient has pneumothorax in their chest x-ray. Indicate `Absent` if pneumothorax is not seen or if a previously observed pneumothorax is mentioned as resolved.",
        "show_options_in_hint": True,
        "logprobs": True,
        "CoT": True,
    },
    {
        "name": "Fracture",
        "type": "select",
        "options": ["Present", "Absent"],
        "hint": "Indicate `Present` if the report explicitly mentions the patient has rib fracture in their chest x-ray. Indicate `Absent` if rib fracture is not seen.",
        "show_options_in_hint": True,
        "logprobs": True,
        "CoT": True,
    },
]

In [7]:
def process_schema(schema):
    processed_schema = []
    for item in schema:
        hint = f"'{item['name']}'\n"
        if item['show_options_in_hint']:
            hint += "Here are your options and you can explicitly use one of these:\n  - " + "\n  - ".join(f"`{i}`" for i in item['options']) + "\n\n"

        hint += "Hint: " + item['hint']

        processed_schema.append({
            "variable": item['name'],
            "CoT": item['CoT'],
            "hint": hint,
            "type": item['type'],
            "options": item['options'] if item['type'] == "select" else None,
        })

    return processed_schema

In [8]:
process_schema(SCHEMA)

[{'variable': 'Pneumothorax',
  'CoT': True,
  'hint': "'Pneumothorax'\nHere are your options and you can explicitly use one of these:\n  - `Present`\n  - `Absent`\n\nHint: Indicate `Present` if the report explicitly mentions the patient has pneumothorax in their chest x-ray. Indicate `Absent` if pneumothorax is not seen or if a previously observed pneumothorax is mentioned as resolved.",
  'type': 'select',
  'options': ['Present', 'Absent']},
 {'variable': 'Fracture',
  'CoT': True,
  'hint': "'Fracture'\nHere are your options and you can explicitly use one of these:\n  - `Present`\n  - `Absent`\n\nHint: Indicate `Present` if the report explicitly mentions the patient has rib fracture in their chest x-ray. Indicate `Absent` if rib fracture is not seen.",
  'type': 'select',
  'options': ['Present', 'Absent']}]

In [9]:
SYS_PROMPT = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."
USER_PROMPT = \
"""Carefully review the provided chest x-ray report (in the <report> tag). Ensure that each data element is accurately captured.

Here is the report:
<report>
{{report}}
</report>

Please pay attention to the following details:
- Your attention to detail is crucial for maintaining the integrity of the medical records. 
- You should not confabulate information, and if something is not mentioned, you should assume that it is `Absent` unless otherwise stated.
- The report may contain additional information that is not relevant to the requested data elements. Please ignore that information.
- We are interested at findings at the time of scan, not the previous ones, so only consider the impression and findings sections of the report.
- Do not print anything else other than the provided output format.
"""

CoT_TEMPLATE = \
"""I want you to extract the following data element from the report:
{{hint}}

After you provide the data element, I will ask you to provide an explanation and then the final answer.

Now give your initial answer. Then provide a step-by-step expalanation based on the information in the report, using no more than three short sentences. You can use less sentences if needed.\
Try to critically appraise your initial answer, which MIGHT be wrong. Then give me your final answer. Format your answers with this format as:

<answer>
<initial_answer>
initial answer goes here
</initial_answer>
<explanation>
1. your first explanation goes here
2. your second explanation goes here (if needed)
3. your third explanation goes here (if needed)
</explanation>
<final_answer>
final answer goes here
</final_answer>
</answer>
"""

RESPONSE_noCoT_TEMPLATE = \
"""I want you to extract the following data element from the report: 
{{hint}}

Your final answer elements in a valid JSON format and is between <json> and </json> tags:

<json>
{
  "{{variable}}": "answer" 
}
</json>
"""

RESPONSE_CoT_TEMPLATE = \
"""Now, based on your expalanation, provide your final answer elements in a valid JSON format and is between <json> and </json> tags:

<json>
{
  "{{variable}}": "answer" 
}
</json>
"""

ASSISTANT_JSON_PREFILL = \
"""
<json>
{
  "{{variable}}" : \""""

In [10]:
pneumonia_prompt = Prompt(
    system_prompt=SYS_PROMPT,
    user_prompts=[
        USER_PROMPT + CoT_TEMPLATE,
        RESPONSE_CoT_TEMPLATE,
    ],
    response_templates=[
        "<answer>\n<initial_answer>",
        ASSISTANT_JSON_PREFILL,
    ],
    stop_tags=[
        "</answer>",
        "</json>",
    ]
)

In [11]:
pneumonia_prompt

In [12]:
pneumonia_prompt.version

'2024-05-03 19:07:09'

In [13]:
pneumonia_prompt.save("pneumonia.rdp")

In [12]:
Prompt().load("pneumonia.rdp").version

'2024-05-03 18:48:24'

In [15]:
client = OpenAIClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:10001/v1",
    api_key = "EMPTY",
)

In [16]:
engine = RadPrompter(client, pneumonia_prompt)

In [17]:
engine

In [15]:
sample = {
    "report": "This is a sample report.",
    "hint": "This is a hint.",
    "variable": "Pneumonia",
}
engine(sample)

Processing items:   0%|          | 0/1 [00:00<?, ?it/s]

Processing items: 100%|██████████| 1/1 [00:03<00:00,  3.60s/it]

<initial_answer>None</initial_answer>
<explanation>
1. Since the report is blank, there is no information provided to extract any data elements.
2. The report does not contain any relevant information about the patient's condition or findings.
3. Therefore, it is reasonable to assume that all data elements are Absent.
</explanation>
<final_answer>Absent</final_answer>

Absent



[[{'role': 'system',
   'content': 'You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements.'},
  {'role': 'user',
   'content': 'Carefully review the provided chest x-ray report (in the <report> tag). Ensure that each data element is accurately captured.\n\nHere is the report:\n<report>\nThis is a sample report.\n</report>\n\nPlease pay attention to the following details:\n- Your attention to detail is crucial for maintaining the integrity of the medical records. \n- You should not confabulate information, and if something is not mentioned, you should assume that it is `Absent` unless otherwise stated.\n- UNDER NO CIRCUIMSTANCES you should use any of your pre-existing medical knowledge, and you MUST only rely on infromation from the report.\n- The report may contain additional information that is not relevant to the requested data elements. Please ignore that information.\n- We are interested at findings at the time of scan